In [1]:
# Import necessary libraries;
import numpy as np
from numpy import mean
from numpy import std


import pandas as pd

import matplotlib.pyplot as plt
from scipy import stats

# use seaborn plotting defaults
import seaborn as sns; sns.set_style('white')

from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)


from mpl_toolkits.mplot3d import Axes3D
from numpy import genfromtxt
import sklearn.metrics as metrics
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.datasets import load_digits, make_blobs
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, plot_confusion_matrix
from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, StandardScaler 
from sklearn.preprocessing import RobustScaler, Normalizer, QuantileTransformer, PowerTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
from sklearn import tree
import plotly.express as px

In [2]:
gammaData = pd.read_csv('telescope_data.csv')
del gammaData['Unnamed: 0']
gammaData.head()

X = gammaData.iloc[:,0:9].values
y = gammaData.iloc[:,10].values
gammaData.head()

,fLength,fWidth,fSize,fConc,fConc1,fAsym,fM3Long,fM3Trans,fAlpha,fDist,class
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,g
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,g
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,g
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,g
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,g


In [3]:
from sklearn.preprocessing import LabelEncoder
encodeLabel = LabelEncoder()
y = encodeLabel.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.75, random_state=1738)


# RANDOM FORREST TRIAL ONE ON GAMMA DATASET

In [4]:
import warnings
warnings.filterwarnings('ignore')

ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = gammaData.sample(n = 5000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:9].values
    ySet = random5000DataPoints.iloc[:,10].values
    ySet = encodeLabel.fit_transform(ySet)
    
    pipe = Pipeline([('std', StandardScaler()),('classifier', RandomForestClassifier(n_jobs = -1))])

    search_space = [
                {
                 
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': ['auto'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']},
                
                {
                 
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': [ 'sqrt'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']},
                
                {
                
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': ['log2'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']
                
                }
                    ]
                
                

    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_1 = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL ONE RESULTS ---------------------------------------------")

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    8.4s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:   10.7s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:   13.3s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:   16.7s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:   20.4s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:   24.5s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:   27.4s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:   30.1s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   34.2s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:   37.6s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:   42.6s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

---------------------------------TRIAL ONE RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:  1.3min finished


In [5]:
TRIAL_1.cv_results_

{'mean_fit_time': array([0.80809288, 1.60878334, 3.2728147 , 0.97183585, 1.94056902,
        3.8672256 , 1.10805278, 2.15074935, 4.25045524, 1.13437567,
        2.26684952, 4.46874299, 1.26979175, 2.52867432, 5.09418092,
        1.49588652, 3.08074932, 6.11325626, 1.81516094, 3.47508841,
        6.8545949 , 1.91865015, 3.69527822, 7.38575211, 0.83342609,
        1.75991302, 3.33546867, 1.01066923, 2.00762663, 3.92557631,
        1.11235652, 2.19478745, 4.20571685, 1.15639448, 2.27925982,
        4.64009032, 1.31963482, 2.66309037, 5.28584576, 1.54472804,
        3.13289437, 6.21424398, 1.74359889, 3.35118184, 6.81842318,
        1.78903861, 3.62521734, 7.28216248, 0.84442635, 1.73659325,
        3.29713545, 1.01547332, 1.87301087, 3.68086534, 1.0326879 ,
        2.03474956, 4.13996048, 1.12246532, 2.2623455 , 4.53580065,
        1.29881692, 2.56160302, 5.19656897, 1.64631581, 3.05252495,
        6.03759217, 1.68574991, 3.37041321, 6.56354427, 1.82136602,
        3.49340472, 5.74524126]

In [6]:
print("---------------------------------TRIAL ONE NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_1.cv_results_['rank_test_neg_log_loss']
TRIAL_1.cv_results_['params'][ np.argmin(TRIAL_1.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL ONE NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'entropy',
 'classifier__max_depth': 13,
 'classifier__max_features': 'auto',
 'classifier__n_estimators': 400}

In [7]:
print("---------------------------------TRIAL ONE F1 RESULTS ---------------------------------------------")
TRIAL_1.cv_results_['rank_test_f1_micro']
TRIAL_1.cv_results_['params'][ np.argmin(TRIAL_1.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL ONE F1 RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'gini',
 'classifier__max_depth': 13,
 'classifier__max_features': 'log2',
 'classifier__n_estimators': 100}

In [8]:
print("---------------------------------TRIAL ONE ROC AUC RESULTS ---------------------------------------------")
TRIAL_1.cv_results_['rank_test_roc_auc_ovo']
TRIAL_1.cv_results_['params'][ np.argmin(TRIAL_1.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL ONE ROC AUC RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'entropy',
 'classifier__max_depth': 13,
 'classifier__max_features': 'auto',
 'classifier__n_estimators': 400}

In [9]:
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_1.cv_results_['params'][ np.argmin(TRIAL_1.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 1 RF using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_1.cv_results_['params'][ np.argmin(TRIAL_1.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 1 RF using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_1.cv_results_['params'][ np.argmin(TRIAL_1.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 1 RF using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")


------------------------------------------------------------------------------------------------
Accuracy of Trial 1 RF using best NEG LOG LOSS hyperparameters :0.8679284963196635
------------------------------------------------------------------------------------------------
Accuracy of Trial 1 RF using best F1 hyperparameters :0.8668068699614441
------------------------------------------------------------------------------------------------
Accuracy of Trial 1 RF using best ROC_AUC hyperparameters :0.8682089029092184
------------------------------------------------------------------------------------------------


# RANDOM FOREST TRIAL TWO ON GAMMA DATASET

In [10]:
import warnings
warnings.filterwarnings('ignore')

ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = gammaData.sample(n = 5000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:9].values
    ySet = random5000DataPoints.iloc[:,10].values
    ySet = encodeLabel.fit_transform(ySet)
    pipe = Pipeline([('std', StandardScaler()),('classifier', RandomForestClassifier(n_jobs = -1))])

    search_space = [
                {
                 
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': ['auto'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']},
                
                {
                 
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': [ 'sqrt'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']},
                
                {
                
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': ['log2'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']
                
                }
                    ]
                
                

    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_2 = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL TWO RESULTS ---------------------------------------------")

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    6.1s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    8.6s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:   11.4s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:   14.7s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:   18.7s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:   22.7s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:   25.6s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:   28.2s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   32.4s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:   35.5s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:   40.7s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

---------------------------------TRIAL TWO RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:  1.2min finished


In [11]:
TRIAL_2.cv_results_

{'mean_fit_time': array([0.78897815, 1.62790012, 3.39572015, 0.93892684, 1.97169538,
        4.01675467, 1.11495862, 2.16346078, 4.37876558, 1.19282556,
        2.43249168, 4.78161192, 1.35116205, 2.70542665, 5.31847401,
        1.5670475 , 3.10887351, 6.23165908, 1.77762885, 3.5830811 ,
        6.96619072, 1.8810174 , 3.72530365, 7.41307526, 0.82040539,
        1.61248693, 3.40082483, 0.99715757, 1.93946819, 3.75683093,
        1.06951938, 2.27025228, 4.33352675, 1.20683765, 2.28996906,
        4.5131815 , 1.27639742, 2.54228644, 5.33238578, 1.57665591,
        3.22177076, 6.38939476, 1.86830678, 3.423244  , 6.92515535,
        1.81506119, 3.60339885, 7.34481668, 0.82581043, 1.63240352,
        3.31374979, 0.96863313, 1.91134391, 3.8102767 , 1.094241  ,
        2.10130739, 4.22593431, 1.14538484, 2.27445579, 4.48135386,
        1.24627151, 2.54268689, 5.27313504, 1.56684709, 3.25640049,
        6.23105884, 1.74129739, 3.43715582, 6.61679044, 1.82617068,
        3.5192266 , 5.67367988]

In [12]:
print("---------------------------------TRIAL TWO NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_2.cv_results_['rank_test_neg_log_loss']
TRIAL_2.cv_results_['params'][ np.argmin(TRIAL_2.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL TWO NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'gini',
 'classifier__max_depth': 13,
 'classifier__max_features': 'auto',
 'classifier__n_estimators': 200}

In [13]:
print("---------------------------------TRIAL TWO F1 RESULTS ---------------------------------------------")
TRIAL_2.cv_results_['rank_test_f1_micro']
TRIAL_2.cv_results_['params'][ np.argmin(TRIAL_2.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL TWO F1 RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'entropy',
 'classifier__max_depth': 13,
 'classifier__max_features': 'auto',
 'classifier__n_estimators': 400}

In [14]:
print("---------------------------------TRIAL TWO ROC AUC RESULTS ---------------------------------------------")
TRIAL_2.cv_results_['rank_test_roc_auc_ovo']
TRIAL_2.cv_results_['params'][ np.argmin(TRIAL_2.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL TWO ROC AUC RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'entropy',
 'classifier__max_depth': 13,
 'classifier__max_features': 'auto',
 'classifier__n_estimators': 400}

In [15]:

print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_2.cv_results_['params'][ np.argmin(TRIAL_2.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 2 RF using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_2.cv_results_['params'][ np.argmin(TRIAL_2.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 2 RF using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_2.cv_results_['params'][ np.argmin(TRIAL_2.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 2 RF using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")


------------------------------------------------------------------------------------------------
Accuracy of Trial 2 RF using best NEG LOG LOSS hyperparameters :0.8677181913774974
------------------------------------------------------------------------------------------------
Accuracy of Trial 2 RF using best F1 hyperparameters :0.8664563617245006
------------------------------------------------------------------------------------------------
Accuracy of Trial 2 RF using best ROC_AUC hyperparameters :0.8679985979670523
------------------------------------------------------------------------------------------------


# RANDOM FOREST TRIAL THREE ON GAMMA DATASET

In [16]:
import warnings
warnings.filterwarnings('ignore')

ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = gammaData.sample(n = 5000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:9].values
    ySet = random5000DataPoints.iloc[:,10].values
    ySet = encodeLabel.fit_transform(ySet)
    
    pipe = Pipeline([('std', StandardScaler()),('classifier', RandomForestClassifier(n_jobs = -1))])

    search_space = [
                {
                 
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': ['auto'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']},
                
                {
                 
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': [ 'sqrt'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']},
                
                {
                
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': ['log2'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']
                
                }
                    ]
                
    #'precision_micro','roc_auc_ovo','jaccard','neg_log_loss'] JACCARD DOES NOT WORK FOR NON BINARY?
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_3 = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL THREE RESULTS ---------------------------------------------")

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    8.6s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:   11.4s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:   14.8s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:   18.7s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:   22.7s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:   25.6s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:   28.5s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   32.5s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:   35.7s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:   40.6s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

---------------------------------TRIAL THREE RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:  1.2min finished


In [17]:
TRIAL_3.cv_results_

{'mean_fit_time': array([0.8032907 , 1.6816462 , 3.39071531, 0.97824149, 1.91214433,
        3.83039398, 1.13007169, 2.179074  , 4.28718681, 1.25037546,
        2.40216556, 4.71795735, 1.29301181, 2.77768869, 5.15823574,
        1.58286114, 3.20645733, 6.39950342, 1.76511784, 3.50981846,
        7.02334008, 1.88552127, 3.73050804, 7.62735929, 0.85873857,
        1.79104013, 3.39932332, 1.04039464, 1.97029448, 3.70788841,
        1.06251354, 2.11451821, 4.45863428, 1.12796993, 2.38575153,
        4.61226659, 1.35046158, 2.59443121, 5.30946593, 1.66433115,
        3.17703218, 6.05280471, 1.69035382, 3.37710385, 6.6268991 ,
        1.83147483, 3.58748527, 7.11591973, 0.7994874 , 1.60808301,
        3.18483877, 0.94040866, 1.88261895, 3.72440271, 1.01807532,
        2.06827893, 4.21692619, 1.14148154, 2.48313546, 4.70854907,
        1.33104458, 2.57161169, 5.20257406, 1.50199161, 3.02069774,
        6.06861868, 1.7550096 , 3.43715572, 6.61989326, 1.80595293,
        3.62541776, 5.72192097]

In [18]:
print("---------------------------------TRIAL THREE NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_3.cv_results_['rank_test_neg_log_loss']
TRIAL_3.cv_results_['params'][ np.argmin(TRIAL_3.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL THREE NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'entropy',
 'classifier__max_depth': 13,
 'classifier__max_features': 'sqrt',
 'classifier__n_estimators': 200}

In [19]:
print("---------------------------------TRIAL THREE F1 RESULTS ---------------------------------------------")
TRIAL_3.cv_results_['rank_test_f1_micro']
TRIAL_3.cv_results_['params'][ np.argmin(TRIAL_3.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL THREE F1 RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'gini',
 'classifier__max_depth': 13,
 'classifier__max_features': 'log2',
 'classifier__n_estimators': 100}

In [20]:
print("---------------------------------TRIAL THREE ROC AUC RESULTS ---------------------------------------------")
TRIAL_3.cv_results_['rank_test_roc_auc_ovo']
TRIAL_3.cv_results_['params'][ np.argmin(TRIAL_3.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL THREE ROC AUC RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'entropy',
 'classifier__max_depth': 13,
 'classifier__max_features': 'auto',
 'classifier__n_estimators': 200}

In [21]:

print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_3.cv_results_['params'][ np.argmin(TRIAL_3.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 3 RF using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_3.cv_results_['params'][ np.argmin(TRIAL_3.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 3 RF using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_3.cv_results_['params'][ np.argmin(TRIAL_3.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 3 RF using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))


------------------------------------------------------------------------------------------------
Accuracy of Trial 3 RF using best NEG LOG LOSS hyperparameters :0.8680686996144409
------------------------------------------------------------------------------------------------
Accuracy of Trial 3 RF using best F1 hyperparameters :0.866596565019278
------------------------------------------------------------------------------------------------
Accuracy of Trial 3 RF using best ROC_AUC hyperparameters :0.8683491062039957


# RANDOM FOREST TRIAL FOUR ON GAMMA DATASET

In [22]:
import warnings
warnings.filterwarnings('ignore')

ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = gammaData.sample(n = 5000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:9].values
    ySet = random5000DataPoints.iloc[:,10].values
    ySet = encodeLabel.fit_transform(ySet)
    
    pipe = Pipeline([('std', StandardScaler()),('classifier', RandomForestClassifier(n_jobs = -1))])

    search_space = [
                {
                 
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': ['auto'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']},
                
                {
                 
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': [ 'sqrt'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']},
                
                {
                
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': ['log2'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']
                
                }
                    ]
                
    #'precision_micro','roc_auc_ovo','jaccard','neg_log_loss'] JACCARD DOES NOT WORK FOR NON BINARY?
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_4 = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL FOUR RESULTS ---------------------------------------------")

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    6.1s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    8.9s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:   11.7s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:   15.1s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:   18.9s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:   22.7s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:   25.4s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:   28.1s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   32.5s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:   36.1s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:   40.7s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

---------------------------------TRIAL FOUR RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:  1.2min finished


In [23]:
TRIAL_4.cv_results_

{'mean_fit_time': array([0.82330799, 1.58436222, 3.36389289, 1.00996861, 1.91124277,
        3.88404069, 1.09864464, 2.18087597, 4.47995276, 1.29141045,
        2.53027563, 5.04613872, 1.38579168, 2.78115411, 5.60762248,
        1.71517506, 3.16372976, 6.06871967, 1.67243805, 3.41613789,
        6.66303015, 1.79254127, 3.56386476, 7.2037951 , 0.80769472,
        1.58726511, 3.35028143, 0.95902486, 1.9879096 , 4.07250223,
        1.08263121, 2.20249405, 4.37876568, 1.31473079, 2.46391878,
        4.75438843, 1.37798495, 2.6765017 , 5.2388052 , 1.53311858,
        3.16071811, 6.22155023, 1.72798595, 3.36929779, 6.69155464,
        1.79865699, 3.5234302 , 7.06737781, 0.79448338, 1.60598135,
        3.20705843, 0.95301948, 1.91074328, 3.69487729, 1.06911941,
        2.09420114, 4.13285403, 1.10424962, 2.20289435, 4.45092788,
        1.25898242, 2.50935788, 5.02041731, 1.49600105, 3.02600236,
        5.98314514, 1.65952697, 3.32916317, 6.45975566, 1.78053126,
        3.46559768, 5.5635848 ]

In [24]:
print("---------------------------------TRIAL FOUR NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_4.cv_results_['rank_test_neg_log_loss']
TRIAL_4.cv_results_['params'][ np.argmin(TRIAL_4.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL FOUR NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'entropy',
 'classifier__max_depth': 13,
 'classifier__max_features': 'sqrt',
 'classifier__n_estimators': 400}

In [25]:
print("---------------------------------TRIAL FOUR F1 RESULTS ---------------------------------------------")
TRIAL_4.cv_results_['rank_test_f1_micro']
TRIAL_4.cv_results_['params'][ np.argmin(TRIAL_4.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL FOUR F1 RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'gini',
 'classifier__max_depth': 13,
 'classifier__max_features': 'log2',
 'classifier__n_estimators': 200}

In [26]:
print("---------------------------------TRIAL FOUR ROC AUC RESULTS ---------------------------------------------")
TRIAL_4.cv_results_['rank_test_roc_auc_ovo']
TRIAL_4.cv_results_['params'][ np.argmin(TRIAL_4.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL FOUR ROC AUC RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'gini',
 'classifier__max_depth': 13,
 'classifier__max_features': 'auto',
 'classifier__n_estimators': 200}

In [27]:

print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_4.cv_results_['params'][ np.argmin(TRIAL_4.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 4 RF using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_4.cv_results_['params'][ np.argmin(TRIAL_4.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 4 RF using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_4.cv_results_['params'][ np.argmin(TRIAL_4.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 4 RF using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")


------------------------------------------------------------------------------------------------
Accuracy of Trial 4 RF using best NEG LOG LOSS hyperparameters :0.8671573781983877
------------------------------------------------------------------------------------------------
Accuracy of Trial 4 RF using best F1 hyperparameters :0.8673676831405538
------------------------------------------------------------------------------------------------
Accuracy of Trial 4 RF using best ROC_AUC hyperparameters :0.8689099193831055
------------------------------------------------------------------------------------------------


# RANDOM FOREST TRIAL FIVE ON GAMMA  DATASET

In [28]:
import warnings
warnings.filterwarnings('ignore')

ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = gammaData.sample(n = 5000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:9].values
    ySet = random5000DataPoints.iloc[:,10].values
    ySet = encodeLabel.fit_transform(ySet)
    
    pipe = Pipeline([('std', StandardScaler()),('classifier', RandomForestClassifier(n_jobs = -1))])

    search_space = [
                {
                 
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': ['auto'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']},
                
                {
                 
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': [ 'sqrt'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']},
                
                {
                
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': ['log2'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']
                
                }
                    ]
                
    #'precision_micro','roc_auc_ovo','jaccard','neg_log_loss'] JACCARD DOES NOT WORK FOR NON BINARY?
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_5 = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL FIVE RESULTS ---------------------------------------------")

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    8.2s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:   10.9s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:   14.2s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:   17.9s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:   21.8s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:   24.5s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:   27.0s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   31.3s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:   34.3s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:   39.2s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

---------------------------------TRIAL FIVE RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:  1.2min finished


In [29]:
TRIAL_5.cv_results_

{'mean_fit_time': array([0.77937045, 1.57435389, 3.18553886, 0.93670521, 1.91274481,
        3.77084303, 1.09474168, 2.19148474, 4.27357521, 1.12917099,
        2.29927721, 4.60165753, 1.28710675, 2.59853473, 5.23440132,
        1.5606421 , 3.04672008, 5.99125247, 1.67624145, 3.32415862,
        6.60257826, 1.76802039, 3.50881767, 7.12592821, 0.8113977 ,
        1.66222949, 3.27471614, 0.93350267, 1.86720595, 3.72160034,
        1.04619951, 2.04505849, 4.22513371, 1.13357501, 2.36273227,
        4.57313266, 1.33024344, 2.54729075, 5.09918513, 1.5011909 ,
        3.02442145, 6.29040937, 1.68775177, 3.54554896, 7.24432993,
        2.11491895, 3.82739124, 7.13013177, 0.83192677, 1.61598969,
        3.20695777, 0.92709708, 1.86350298, 3.71169205, 1.05420651,
        2.05366607, 4.17989478, 1.10985456, 2.24042678, 4.48595786,
        1.26098437, 2.50995822, 5.084973  , 1.49888887, 2.95534167,
        6.00466385, 1.66373048, 3.28382411, 6.45375018, 1.75450883,
        3.41013269, 5.42506585]

In [30]:
print("---------------------------------TRIAL FIVE NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_5.cv_results_['rank_test_neg_log_loss']
TRIAL_5.cv_results_['params'][ np.argmin(TRIAL_5.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL FIVE NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'entropy',
 'classifier__max_depth': 13,
 'classifier__max_features': 'auto',
 'classifier__n_estimators': 400}

In [31]:
print("---------------------------------TRIAL FIVE F1 RESULTS ---------------------------------------------")
TRIAL_5.cv_results_['rank_test_f1_micro']
TRIAL_5.cv_results_['params'][ np.argmin(TRIAL_5.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL FIVE F1 RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'gini',
 'classifier__max_depth': 13,
 'classifier__max_features': 'auto',
 'classifier__n_estimators': 100}

In [32]:
print("---------------------------------TRIAL FIVE ROC AUC RESULTS ---------------------------------------------")
TRIAL_5.cv_results_['rank_test_roc_auc_ovo']
TRIAL_5.cv_results_['params'][ np.argmin(TRIAL_5.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL FIVE ROC AUC RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'entropy',
 'classifier__max_depth': 13,
 'classifier__max_features': 'auto',
 'classifier__n_estimators': 400}

In [33]:

print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_5.cv_results_['params'][ np.argmin(TRIAL_5.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 5 RF using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_5.cv_results_['params'][ np.argmin(TRIAL_4.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 5 RF using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_5.cv_results_['params'][ np.argmin(TRIAL_5.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 5 RF using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")


------------------------------------------------------------------------------------------------
Accuracy of Trial 5 RF using best NEG LOG LOSS hyperparameters :0.8671573781983877
------------------------------------------------------------------------------------------------
Accuracy of Trial 5 RF using best F1 hyperparameters :0.8677882930248861
------------------------------------------------------------------------------------------------
Accuracy of Trial 5 RF using best ROC_AUC hyperparameters :0.8682790045566071
------------------------------------------------------------------------------------------------
